In [2]:
import pandas as pd

In [5]:
# Load the datasets
awards_players_df = pd.read_csv('../../dataset/awards_players.csv')
coaches_df = pd.read_csv('../../dataset/coaches.csv')
players_df = pd.read_csv('../../dataset/players.csv')
players_teams_df = pd.read_csv('../../dataset/players_teams.csv')
series_post_df = pd.read_csv('../../dataset/series_post.csv')
teams_df = pd.read_csv('../../dataset/teams.csv')
teams_post_df = pd.read_csv('../../dataset/teams_post.csv')

# Remove useless columns from the datasets
awards_players_df = awards_players_df.drop(columns=['lgID'])
players_df = players_df.drop(columns=['firstseason', 'lastseason', 'deathDate'])
coaches_df = coaches_df.drop(columns=['lgID'])
series_post_df = series_post_df.drop(columns=['lgIDLoser', 'lgIDWinner'])
teams_post_df = teams_post_df.drop(columns=['lgID'])
teams_df = teams_df.drop(columns=['lgID', 'divID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB'])
players_teams_df = players_teams_df.drop(columns=['lgID'])

# Display the first few rows of each cleaned DataFrame (optional)
print(awards_players_df.head())
print(players_df.head())
print(coaches_df.head())
print(series_post_df.head())
print(teams_post_df.head())
print(teams_df.head())
print(players_teams_df.head())


     playerID                               award  year
0  thompti01w  All-Star Game Most Valuable Player     1
1  leslili01w  All-Star Game Most Valuable Player     2
2  leslili01w  All-Star Game Most Valuable Player     3
3  teaslni01w  All-Star Game Most Valuable Player     4
4  swoopsh01w  All-Star Game Most Valuable Player     6
        bioID  pos  height  weight            college            collegeOther  \
0  abrahta01w    C    74.0     190  George Washington                     NaN   
1  abrossv01w    F    74.0     169        Connecticut                     NaN   
2  adairje01w    C    76.0     197  George Washington                     NaN   
3  adamsda01w  F-C    73.0     239          Texas A&M  Jefferson College (JC)   
4  adamsjo01w    C    75.0     180         New Mexico                     NaN   

    birthDate  
0  1975-09-27  
1  1980-07-09  
2  1986-12-19  
3  1989-02-19  
4  1981-05-24  
      coachID  year tmID  stint  won  lost  post_wins  post_losses
0  adamsmi01w 

In [5]:
# 

     playerID                               award  year  lgID
0  thompti01w  All-Star Game Most Valuable Player     1  WNBA
1  leslili01w  All-Star Game Most Valuable Player     2  WNBA
2  leslili01w  All-Star Game Most Valuable Player     3  WNBA
3  teaslni01w  All-Star Game Most Valuable Player     4  WNBA
4  swoopsh01w  All-Star Game Most Valuable Player     6  WNBA
